# Artificial Intelligence
A Jupyter Notebook exploring deep learning.

In [52]:
from __future__ import print_function

import plaidml.keras
plaidml.keras.install_backend()

from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

path = get_file(
#     'nietzsche.txt',
#     origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
    "abc.txt", origin='http://localhost:5000/abc.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

corpus length: 25400
total chars: 56
nb sequences: 8454
Vectorization...
Build model...
Epoch 1/60
8454/8454 [==============================] - 4s 477us/step - loss: 2.9853

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "nder hamilton; al capone and bugs moran;"
nder hamilton; al capone and bugs moran; fararanarinararanararann bararar anarararararararararararan arareranganarand fare wararararararararanarararanararararinarararand fanarararararararinn arararing an anranararand fararanaran an anaran anarang anthere an an an anaranarararararararandafafarararananararararare fofararandafarararararar anarararararararararerararanararararan aan ararararareranarararararinar ananararanarangafare fafararar
----- diversity: 0.5
----- Generating with seed: "nder hamilton; al capone and bugs moran;"
nder hamilton; al capone and bugs moran; and aacanar arincainasrararar fancas fabaraf arana anpasaracathlmffevin arinantananlalain friniear fadans thinl fa fanapisk

it has been revised down about $2 billionlly verose mis anh -anend an int in morter tho n woryer covthers on the angentere vroevloge are fronced anoupparn.

nu bank mener nns.

the e the s ate lain, fereenars and the nuod.

the noctens, llces to and reresems an fram ave sedes't of pepivas aroucer aadeud of srmeaa to lovenorald in 2uriea anotoung's andovemmenn, tho incentare oncebt and froven ander now easter in ar lromas enare on wornce
----- diversity: 1.2
----- Generating with seed: "it has been revised down about $2 billio"
it has been revised down about $2 billions in wours atsarey an 2teat ougoap,y anerianan.

ret" and nat on the nuroning"p2)
nand sorepted on wioudions froven rone.

ataverrus -ang arimorens
in 
rontwone, asoreal rorarted foro"ukeden ann mura2t anisiang fiofrere ic biope painsur, counslictistongoliun,, ealad0e batoore undovanmen, he apduplend on arey a dast.eal ofh aeess apfraislino in toe ont3s, mun corurino nadency a ane novere ncrone, 
Epoch 5/60
8454/8454 [====

on of prints and drawings by the two artors the nurpaning the number finang the number the nown the nang the number the number the nationt the number the naning the tho fine conting the nown the number caning the number the numbers an the number the strone the numbers sting the numbers and the number the nown the numbers and the number the nurpaning the sting the number filice the number the number the number contion the number the numb
----- diversity: 0.5
----- Generating with seed: "on of prints and drawings by the two art"
on of prints and drawings by the two artore bit in and
inture diting the ning tion on the niting the numbiof the number than sting the not and the nurper to bi andanigate becound soring the nusben nushe., and intertions the number mane and the number the saining tho gatits liont and with a stown tho n with repicans suine.

the nort an and inthan on the numbers the nurpent sousd the numbers and the number mation des innation the numbion 
----- diversity: 1.0
-----

year budget forecasts a surplus of $5 billio sibe likand how envine wide nure dern efcours covppices a gothe be off the n wire aroming noreit, whe he tho s a ent a forealbor in lied of who fe-omaning — ivel of chonge aublegode with from "wer file a patlleaver in hov to leker moter in the relutectows evrigates, in tho pcragiss ea hownded bit a sporeaty cropequectioniin fileteavion brome the norcen agriden evo her government
is foll whill
----- diversity: 1.2
----- Generating with seed: "year budget forecasts a surplus of $5 bi"
year budget forecasts a surplus of $5 billiol of tho c inisveys empses,t and the relolage f3locas on the ehusblee sime to mavili vasseun seralien in inpledey rivatiog dor a hass of inplation aasmend ur years avel goe bill conat ov cilouce bo 21,2 powire the loteay a poill saibellipd the number reateff, wrom handay's whalld mavinn for ic witing the sgever likito ficas ous inforealion of halibl auld wos expectay ssomsat-and bes livat ir i
Epoch 12/60
8454/8454 [===

 paris in 1906 — picasso, an angsty 24-y aroning to be the belard and somore ong to be the stralied and wet not a dos and intormenion derentery mire the ont om penroctown dationalmman effectstramint in the other to ds, act of the onoment of cromed of the nortaymen the one was mast the ont om procesting in the onoming to art plotembit of the straling to be the hast the stragement of the ont of ker in hor be of a contry mare the one basme
----- diversity: 0.5
----- Generating with seed: " paris in 1906 — picasso, an angsty 24-y"
 paris in 1906 — picasso, an angsty 24-y arouth of the nortanmen to madicts a bit bleckesses tare an ortivmens for the straming is derections."

mmang, were tho soupdy motement of the nortay, moromses, and bromaten, and aldown manistred and with her oftom numbers on 2019.
noroman — eadrous on bit in a someng to be the namention on the narding to to comesting and bregher of the ont om prowed of 2019.

the nament progicentry marn spatting
----- diversity: 1.0
-----

...

am gather condequrnds, greant sleend.

the fremehtle, and whe cholles hot iment intermicter fra
----- diversity: 1.2
----- Generating with seed: "sang in cabarets before coming to france"
sang in cabarets before coming to france intels, but it f eeplay arom stares and be nat tors wile alpour cintruce shill on hir had ble gate thise that is guld atprespure manister in mate surges are cenfore avsum of the nsw, ove gall is uptly foreceus on than fall 06 sigilition spouph-es 197-am oricant suld be nuthe was alpotlent ngers gover ment's swip
thane.

the lode antis, 2014.

of hil pailly beann wome .
ine said.

"t weal spedchin
Epoch 45/60
8454/8454 [==============================] - 4s 441us/step - loss: 0.2696

----- Generating text after Epoch: 44
----- diversity: 0.2
----- Generating with seed: "fore shooting him.

he has been taken to"
fore shooting him.

he has been taken to reduped through lud derecent still of crolles in the comiath, has foult in bres has been chiteritso and shithinal

starred in jacques rivette's 1966 film le forf newy rears.

masiints, she this last were dange of 2018-2 be the crapictor the muching abrustuned this monviro ppectidg der hap bregncustung is pundly enver of tho couns and "
pections."
 was in kogrialion has laskenmer of the decreser of tho coups, a she number the souve to ke picasso is pascection in fara like been mite weish actuaslions for a coveriated acturemesional madage of the north
----- diversity: 1.0
----- Generating with seed: "starred in jacques rivette's 1966 film l"
starred in jacques rivette's 1966 film le move nebrs,s, audhoonden'aged the colle buonned." thro this movert years."
kne backospioned that for a coverould to be liginal for the ns.uved act of the "to, lithiove to andistravion in kelly dramittem, spiunt to the down on australice baspay'r governmentur beckn was.2201me pilared actibnal efort stave sce picasso train, south-on bromblign foreare the tow relies. promicastina, previously survey
----- diversity: 1.2
-----

cals lovern she said sued the ns, whim ing sprued if artally more the nextion bisner an expicas almes to be two ictours maond reast for the nsti-vanfer a yepitios ampecrousdon, with crome ther it mome the blage manitios, piparces like thin, a was stide bater in jestifn previously dermide mugher trem, sp
Epoch 52/60
8454/8454 [==============================] - 4s 512us/step - loss: 0.2369

----- Generating text after Epoch: 51
----- diversity: 0.2
----- Generating with seed: "roken.

brisbane is expected to hit 39 d"
roken.

brisbane is expected to hit 39 dermese and with were vaniter and has dored gow who chisemiof thit park.

the numbers and kell pira. lionher the evectous thit is australied.

the weve botell pordected conner jonnidery in the government eldes adand edpina.
prome the hore ding norses has lagest's for the $1, having velled the straviesting this ship
twome eboul key plive boret boon a ppencencing der the government evendsting through
----- diversity: 0.5
----- Generating

is the first exhibition in australia to be lithers frem it was nowdanhing to coves are dang forecconts, with cromiged swidg in sound be kangela in the numbar id gatlly nordathive film ie breg the lother fores and beskeand per le othe bold theirs. in the foom vellidally realddr, was could ie breg the lugh luge from a be lab5er come fate be koin weel broke bluk thina.

mas, with sleed foom the conven and enverally nordathian dedad id gatl
----- diversity: 1.0
----- Generating with seed: "is the first exhibition in australia to "
is the first exhibition in australia to lown mears.

mat-is gattoral yearhit lather a somenfer now relled tovey tove nading to would ienortimate bates acout condition of morome to reducen while had be nuthe relativaly foreccont for the ns sorthing, by nagits of the ns,, wirk fert ther pled tho chould in the nurses and bedaraing for the ntuation was fice was abee broke bililican dedlodmands almestion, tho chite but it flom gape bougen ar
----- diversity: 1.2
-----

nurk.teroun, and picasso twhe fide a and quadrmbate bustugeer in a rearblina.

armelued art in der had agen maniefire she shipfine abeunst,2
Epoch 59/60
8454/8454 [==============================] - 4s 453us/step - loss: 0.2202

----- Generating text after Epoch: 58
----- diversity: 0.2
----- Generating with seed: "australians," mr mcgowan said recently o"
australians," mr mcgowan said recently of from tho east velle have wine batila an in has lake was alpeard.

he was alpotlation her beee and proded.
phent and hors in ascentrowan s pects ngo was luke batirs a pastalion in frenses in kell yer have said.

moluce has been a betlee chorome thit moremares stare lake their in respuchine said.

"t whel bat the stuad as well, actomen ore was a poptraliby foreccants: the crome the bother in art s
----- diversity: 0.5
----- Generating with seed: "australians," mr mcgowan said recently o"
australians," mr mcgowan said recently of from tho east velle hat ble that lasd aconition her be the governme